Libraries

In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import pickle
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Ingestion

In [2]:
# Data
path = r"../data/raw/germanCredit.csv"
df_gc = pd.read_csv(path)
df = df_gc.copy()
df_gc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         1000 non-null   int64 
 1   checkin_acc        1000 non-null   object
 2   duration           1000 non-null   int64 
 3   credit_history     1000 non-null   object
 4   amount             1000 non-null   int64 
 5   savings_acc        1000 non-null   object
 6   present_emp_since  1000 non-null   object
 7   inst_rate          1000 non-null   int64 
 8   personal_status    1000 non-null   object
 9   residing_since     1000 non-null   int64 
 10  age                1000 non-null   int64 
 11  inst_plans         1000 non-null   object
 12  num_credits        1000 non-null   int64 
 13  job                1000 non-null   object
 14  status             1000 non-null   int64 
dtypes: int64(8), object(7)
memory usage: 117.3+ KB


Payload

In [3]:
df_check = df[234:235]
out = df_check.to_json(orient='records')[1:-1].replace('},{', '} {')
with open('../assets/payload.json', 'w') as f:
    f.write(out)

In [14]:
payload = {
  "Unnamed: 0": 234,
  "checkin_acc": "A14",
  "duration": 4,
  "credit_history": "A34",
  "amount": 1544,
  "savings_acc": "A61",
  "present_emp_since": "A74",
  "inst_rate": 2,
  "personal_status": "A93",
  "residing_since": 1,
  "age": 42,
  "inst_plans": "A143",
  "num_credits": 3,
  "job": "A172",
  
}

#"status": 0

In [15]:
data = pd.DataFrame([payload])

Encoding

In [16]:
with open("../bin/oneHotEncoder.bin", "rb") as f_in:
    enc = pickle.load(f_in)

In [17]:
from src.featureEngineering import encode
X_en = encode(data = data,
              encoder = enc)

Infererence

In [18]:
with open("../bin/clf_log1.bin", "rb") as f_in:
    model = pickle.load(f_in)

In [23]:
y_prob = model.predict_proba(X_en)[0][0]
y_prob

0.9696953078247862

In [25]:
int(y_prob >= 0.5)

1

Inference fucntion

In [29]:
with open("../bin/oneHotEncoder.bin", "rb") as f_in:
    enc = pickle.load(f_in)

with open("../bin/clf_log1.bin", "rb") as f_in:
    model = pickle.load(f_in)

def predict(payload, encoder, model):
    data = pd.DataFrame([payload])
    X_en = encode(data = data, encoder = enc)

    y_prob = model.predict_proba(X_en)[0,0].round(3)
    verdict = int(y_prob >= 0.5)

    result = {'DECESION': verdict, 'PROBABILITY':y_prob}

    return result

predict(payload=payload, encoder=enc, model=model)      

{'DECESION': 1, 'PROBABILITY': 0.97}